In [46]:
import pandas as pd
import numpy as np
import math
import mil
import tensorflow
from mil.trainer import Trainer
from mil.trainer.trainer import Trainer
from mil.preprocessing import StandarizerBagsList
from mil.bag_representation import MILESMapping
from mil.validators import LeaveOneOut
from mil.metrics import AUC, BinaryAccuracy
from mil.models import SVC


label = []
PSA_train_bags = []
#PSA_train_labels = []
bags = []
veci = []
vecj = []
#df = pd.read_csv('C:/Users/Mavek/Desktop/Magisterka/Prostata_Cechy/MIL/Cechy2/data.txt', sep='\s+', header=None)
df = pd.read_csv(r'data.txt', sep='\s+', header=None)


index = df.values[:,-1]
index = [int(i) for i in index]  # Przekształcenie wartości index na int

feature = df.values[:,:-1]

#feature = pd.to_numeric(feature.flatten(), errors='coerce').reshape(feature.shape)

unikalne_index = np.unique(index)

# grupowane_dane = [list(feature[index == val]) for val in unikalne_index]
# bags = grupowane_dane

grupowane_dane = [list(feature[index == val].tolist()) for val in unikalne_index]
bags = grupowane_dane

#leb = pd.read_csv(r'C:/Users/Mavek/Desktop/Magisterka/Prostata_Cechy/MIL/Cechy2/Label.csv', header=None)
leb = pd.read_csv(r'Label.csv', header=None)

leb = leb[0].str.split(';', expand = True)

leb = leb.apply(pd.to_numeric, errors='coerce')

j = 0

for i in range(len(leb.iloc[:,0])):

    # print(j , " j " , i , " i ")
    # print(unikalne_index[j] , " ui " , leb.iloc[i,0], " leb ")
    if leb.iloc[i,2] <= 3 and unikalne_index[j] == leb.iloc[i,0]:
        

        label.append(1)
        PSA_train_bags.append(bags[j])
        j = j + 1

    elif leb.iloc[i,2] > 3 and unikalne_index[j] == leb.iloc[i,0]:
        

        label.append(0)
        PSA_train_bags.append(bags[j])        
        j = j + 1

    elif unikalne_index[j] > leb.iloc[i,0]:
        pass

    else :
        j = j + 1

    
PSA_train_labels = np.array(label)
#PSA_train_bags = PSA_train_bags.astype(float)




In [47]:
from mil.data.datasets import musk1, musk2, protein, elephant, corel_dogs, \
                              ucsb_breast_cancer, web_recommendation_1, birds_brown_creeper, \
                              mnist_bags


(bags_train, y_train), (bags_test, y_test) = musk1.load()
print(bags_train)
print(PSA_train_bags)


# Count elements in PSA_train_labels that are and are not numbers
numeric_labels = [x for x in PSA_train_labels if isinstance(x, (int, float))]
non_numeric_labels = [x for x in PSA_train_labels if not isinstance(x, (int, float))]

print("Number of numeric elements in PSA_train_labels:", len(numeric_labels))
print("Number of non-numeric elements in PSA_train_labels:", len(non_numeric_labels))

# Get indices, values, and types of elements in PSA_train_labels that are not numbers
non_numeric_labels_info = [(index, x, type(x)) for index, x in enumerate(PSA_train_labels) if not isinstance(x, (int, float))]
print("Indices, values, and types of non-numeric elements in PSA_train_labels:", non_numeric_labels_info)

# Count elements in PSA_train_bags that are and are not numbers
numeric_elements = [item for bag in PSA_train_bags for sublist in bag for item in sublist if isinstance(item, (int, float))]
non_numeric_elements = [item for bag in PSA_train_bags for sublist in bag for item in sublist if not isinstance(item, (int, float))]

print("Number of numeric elements in PSA_train_bags:", len(numeric_elements))
print("Number of non-numeric elements in PSA_train_bags:", len(non_numeric_elements))

# Get indices, values, and types of elements in PSA_train_bags that are not numbers
non_numeric_elements_info = [(bag_index, sublist_index, item_index, item, type(item)) for bag_index, bag in enumerate(PSA_train_bags) for sublist_index, sublist in enumerate(bag) for item_index, item in enumerate(sublist) if not isinstance(item, (int, float))]
print("Indices, values, and types of non-numeric elements in PSA_train_bags:", non_numeric_elements_info)


[[[23, -197, -159, 140, -117, -44, 42, 44, -32, -111, -57, -126, -52, -110, -75, -299, 33, -71, -74, -84, -98, -18, -69, -23, -92, 2, -9, -65, 4, -144, -116, -1, 32, -233, 107, 46, -175, -1, -143, -32, -182, -107, -68, -68, -134, -134, -116, -84, -28, -107, -111, -20, -26, 69, -17, -68, 56, -146, 6, -162, 150, -178, 38, -188, -28, 16, -165, -108, -133, -112, -168, -148, -50, -143, -52, -181, 2, -75, -104, 48, -90, 4, -58, 194, -175, 32, -131, -54, 62, -146, -202, 9, 81, -134, 43, 90, -164, 37, -153, 33, -204, 16, -49, -53, -91, -157, -176, -98, -63, -133, -64, -72, -72, -60, -192, -113, -85, 61, 69, -13, -112, -73, -92, 29, -40, 7, -146, -191, -124, -9, 55, 103, -85, -130, -111, -101, 44, -99, -54, -48, -38, -36, 17, 16, -178, 113, -118, -118, -127, -15, 114, -8, 55, -168, -22, -183, -239, -271, -209, -133, -75, -18, 42, -33, 25, -2], [8, -196, -146, 116, -117, -7, 63, -10, -28, -172, -54, -102, -93, -92, -104, -270, 32, -100, -96, -36, -59, -16, -81, -62, -88, -30, 27, -117, 120, -144

In [53]:

#------------------------------------------------------------------------------------------------------------

# instantiate trainer
PSA_trainer = Trainer()

# preparing trainer
metrics = ['acc', AUC]
model = SVC(kernel='linear', C=1, class_weight='balanced')
pipeline = [('scale', StandarizerBagsList()), ('disc_mapping', MILESMapping())]
PSA_trainer.prepare(model, preprocess_pipeline=pipeline ,metrics=metrics)

# fitting trainer
valid = LeaveOneOut()

PSA_history = PSA_trainer.fit(PSA_train_bags, PSA_train_labels, sample_weights='balanced', validation_strategy=valid, verbose=1)


#print(PSA_history)
print(PSA_history['metrics_val'])

PSA_history_F1 = []  # Dodane: inicjalizacja listy przed pętlą


40/40 [==============================] - 5s 135ms/step - train_accuracy: 1.0000 - train_auc: 1.0000 - val_accuracy: 1.0000 - val_auc: 0.0000e+00
[{'accuracy': 1.0, 'auc': 0.0}, {'accuracy': 0.0, 'auc': 0.0}, {'accuracy': 1.0, 'auc': 0.0}, {'accuracy': 1.0, 'auc': 0.0}, {'accuracy': 1.0, 'auc': 0.0}, {'accuracy': 1.0, 'auc': 0.0}, {'accuracy': 1.0, 'auc': 0.0}, {'accuracy': 1.0, 'auc': 0.0}, {'accuracy': 0.0, 'auc': 0.0}, {'accuracy': 1.0, 'auc': 0.0}, {'accuracy': 1.0, 'auc': 0.0}, {'accuracy': 1.0, 'auc': 0.0}, {'accuracy': 1.0, 'auc': 0.0}, {'accuracy': 1.0, 'auc': 0.0}, {'accuracy': 1.0, 'auc': 0.0}, {'accuracy': 1.0, 'auc': 0.0}, {'accuracy': 1.0, 'auc': 0.0}, {'accuracy': 1.0, 'auc': 0.0}, {'accuracy': 1.0, 'auc': 0.0}, {'accuracy': 1.0, 'auc': 0.0}, {'accuracy': 1.0, 'auc': 0.0}, {'accuracy': 1.0, 'auc': 0.0}, {'accuracy': 0.0, 'auc': 0.0}, {'accuracy': 1.0, 'auc': 0.0}, {'accuracy': 1.0, 'auc': 0.0}, {'accuracy': 1.0, 'auc': 0.0}, {'accuracy': 0.0, 'auc': 0.0}, {'accuracy': 1.0,